In [1]:
from shared_utils import catalog_utils
import pandas as pd
import geopandas as gpd

# Quickly get unique stops in D7 on the SHS

In [2]:
shared_data = catalog_utils.get_catalog(catalog_name='shared_data_catalog')

In [3]:
stops = shared_data.ca_transit_stops.read()

In [4]:
d7_shs = stops.query('district_name.str.contains("07 - Los Angeles") & meters_to_ca_state_highway <= 50')

In [5]:
d7_shs.head(3)

,org_id,agency,stop_id,stop_name,n_routes,route_ids_served,routetypes,n_arrivals,n_hours_in_service,meters_to_ca_state_highway,base64_url,district_name,geometry
533,recoRzMb9zCV1Ta0x,City of El Monte,2394746,Tyler Avenue & Brockway Street,1,6814,3,0,0,23.9,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,07 - Los Angeles / Ventura,POINT (-118.03611 34.06839)
561,recoRzMb9zCV1Ta0x,City of El Monte,2394777,Durfee Avenue & Exline Street,1,6816,3,0,0,8.3,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,07 - Los Angeles / Ventura,POINT (-118.01117 34.06460)
579,recoRzMb9zCV1Ta0x,City of El Monte,2382620,Flair Drive & Telstar Avenue,1,6811,3,21,15,26.1,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,07 - Los Angeles / Ventura,POINT (-118.05736 34.07203)


Where count is greater than 1, agencies share a GTFS feed. We count each stop once per feed to prevent double-counting

In [6]:
agency_groups = d7_shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').agency.unique()
agency_groups = pd.DataFrame(agency_groups).reset_index().rename(columns={'agency':'agency_group'})

In [7]:
df = d7_shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').count()

In [8]:
df = df.reset_index().rename(columns={'agency':'count'})

In [9]:
d7_shs[['base64_url', 'agency']].drop_duplicates().merge(df, on='base64_url').sort_values('count', ascending=False)

,base64_url,agency,count
13,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Camarillo,7
11,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,Ventura County Transportation Commission,7
16,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Ojai,7
15,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Moorpark,7
14,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,Gold Coast Transit District,7
12,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Thousand Oaks,7
17,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Simi Valley,7
6,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,Foothill Transit,2
5,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,City of Duarte,2
2,aHR0cDovL2d0ZnMuZ2lzLmZsaXgudGVjaC9ndGZzX2dlbm...,FlixBus,2


In [10]:
gdf = d7_shs.drop_duplicates(subset=['base64_url', 'stop_id']).merge(agency_groups, on='base64_url')

In [11]:
gdf.agency_group = gdf.agency_group.astype(str)

In [12]:
gdf.to_csv('d7_shs_stops_062025.csv')

In [13]:
from shared_utils import webmap_utils
from calitp_data_analysis import geography_utils

In [14]:
to_map = gdf.to_crs(geography_utils.CA_NAD83Albers_m)
to_map.geometry = to_map.buffer(50)

Simple map to share

In [15]:
import branca

In [16]:
cmap = branca.colormap.step.RdPu_05
cmap.scale(vmin = 0, vmax = 50)

In [17]:
cmap.rgb_bytes_tuple(50)

(122, 1, 119)

In [18]:
to_map['color'] = [(122, 1, 119)] * to_map.shape[0]

In [19]:
webmap_utils.set_state_export(to_map, subfolder='adhoc', filename='d7_stops_062025_v2',
                             map_title="D7 Transit Stops within 50 meters of SHS, June 2025")

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


{'state_dict': {'name': 'null',
  'layers': [{'name': 'D7 Transit Stops within 50 meters of SHS, June 2025',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/adhocd7_stops_062025_v2.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (34.00943475388468, -118.34559241563224),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJENyBUcmFuc2l0IFN0b3BzIHdpdGhpbiA1MCBtZXRlcnMgb2YgU0hTLCBKdW5lIDIwMjUiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2FkaG9jZDdfc3RvcHNfMDYyMDI1X3YyLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbMzQuMDA5NDM0NzUzODg0NjgsIC0xMTguMzQ1NTkyNDE1NjMyMjRdLCAiem9vbSI6IDEzfQ=='}

# Quickly get unique stops in D4 on the SHS

In [20]:
shared_data = catalog_utils.get_catalog(catalog_name='shared_data_catalog')

In [21]:
stops = shared_data.ca_transit_stops.read()

In [22]:
d4_shs = stops.query('district_name.str.contains("04") & meters_to_ca_state_highway <= 50')

In [23]:
d4_shs.head(3)

,org_id,agency,stop_id,stop_name,n_routes,route_ids_served,routetypes,n_arrivals,n_hours_in_service,meters_to_ca_state_highway,base64_url,district_name,geometry
288,recmB4uxrVLRXYF3L,City of Union City,3817727,Mission Blvd & Tamarack Dr,1,4,3,17,17,1.8,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.02261 37.60831)
289,recmB4uxrVLRXYF3L,City of Union City,3817728,Mission Blvd. at Whipple Road,1,4,3,17,17,4.8,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.01981 37.60585)
290,recmB4uxrVLRXYF3L,City of Union City,3817729,Mission Blvd. at F St.,1,4,3,17,17,8.8,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.01760 37.60384)


Where count is greater than 1, agencies share a GTFS feed. We count each stop once per feed to prevent double-counting

In [24]:
agency_groups = d4_shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').agency.unique()
agency_groups = pd.DataFrame(agency_groups).reset_index().rename(columns={'agency':'agency_group'})

In [25]:
df = d4_shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').count()

In [26]:
df = df.reset_index().rename(columns={'agency':'count'})

In [27]:
d4_shs[['base64_url', 'agency']].drop_duplicates().merge(df, on='base64_url').sort_values('count', ascending=False)

,base64_url,agency,count
32,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Solano Transportation Authority,2
2,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Commute.org,2
3,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,City of Menlo Park,2
5,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Cloverdale Transit,2
6,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Sonoma County,2
33,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Solano County Transit,2
29,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"Golden Gate Bridge, Highway and Transportation...",1
23,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,San Mateo County Transit District,1
24,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Capitol Corridor Joint Powers Authority,1
25,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,City of South San Francisco,1


In [28]:
gdf = d4_shs.drop_duplicates(subset=['base64_url', 'stop_id']).merge(agency_groups, on='base64_url')

In [29]:
gdf.agency_group = gdf.agency_group.astype(str)

In [30]:
gdf

,org_id,agency,stop_id,stop_name,n_routes,route_ids_served,routetypes,n_arrivals,n_hours_in_service,meters_to_ca_state_highway,base64_url,district_name,geometry,agency_group
0,recmB4uxrVLRXYF3L,City of Union City,3817727,Mission Blvd & Tamarack Dr,1,4,3,17,17,1.8,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.02261 37.60831),['City of Union City']
1,recmB4uxrVLRXYF3L,City of Union City,3817728,Mission Blvd. at Whipple Road,1,4,3,17,17,4.8,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.01981 37.60585),['City of Union City']
2,recmB4uxrVLRXYF3L,City of Union City,3817729,Mission Blvd. at F St.,1,4,3,17,17,8.8,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.01760 37.60384),['City of Union City']
3,recmB4uxrVLRXYF3L,City of Union City,3817730,Mission Blvd. at Daggett Ave.,1,4,3,17,17,5.9,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.01142 37.59832),['City of Union City']
4,recmB4uxrVLRXYF3L,City of Union City,3819169,Appian Wy & Mission Bl,1,4,3,17,17,36.4,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.00309 37.59163),['City of Union City']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654,rec9RGrya9SjWYp2p,Napa Valley Transportation Authority,89402,Napa Valley College (NB),2,"11, G",3,26,14,23.1,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,04 - Bay Area / Oakland,POINT (-122.27306 38.27361),['Napa Valley Transportation Authority']
1655,recsBfXgev9ICDCY1,Presidio Trust,41492,Van Ness & Union (Drop Off),1,66,3,29,15,0.2,aHR0cHM6Ly9wcmVzaWRpb2J1cy5jb20vZ3Rmcw==,04 - Oakland,POINT (-122.42409 37.79861),['Presidio Trust']
1656,recsBfXgev9ICDCY1,Presidio Trust,41520,Van Ness & Union (Pick Up),1,66,3,29,14,0.8,aHR0cHM6Ly9wcmVzaWRpb2J1cy5jb20vZ3Rmcw==,04 - Oakland,POINT (-122.42389 37.79848),['Presidio Trust']
1657,recXIO3srSe6F77pC,Mission Bay Transportation Management Agency,2335513,Berry Street and King Street,1,6283,3,26,12,12.8,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,04 - Oakland,POINT (-122.39839 37.77176),['Mission Bay Transportation Management Agency']


In [31]:
gdf.to_csv('d4_shs_stops_062025.csv')

In [32]:
from shared_utils import webmap_utils
from calitp_data_analysis import geography_utils

In [33]:
to_map = gdf.to_crs(geography_utils.CA_NAD83Albers_m)
to_map.geometry = to_map.buffer(50)

Simple map to share

In [34]:
import branca

In [35]:
cmap = branca.colormap.step.RdPu_05
cmap.scale(vmin = 0, vmax = 50)

In [36]:
cmap.rgb_bytes_tuple(50)

(122, 1, 119)

In [37]:
to_map['color'] = [(122, 1, 119)] * to_map.shape[0]

In [38]:
webmap_utils.set_state_export(to_map, subfolder='adhoc', filename='d4_stops_062025',
                             map_title="D4 Transit Stops within 50 meters of SHS, June 2025")

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


{'state_dict': {'name': 'null',
  'layers': [{'name': 'D4 Transit Stops within 50 meters of SHS, June 2025',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/adhocd4_stops_062025.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (37.81827807013203, -122.34206361619229),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJENCBUcmFuc2l0IFN0b3BzIHdpdGhpbiA1MCBtZXRlcnMgb2YgU0hTLCBKdW5lIDIwMjUiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2FkaG9jZDRfc3RvcHNfMDYyMDI1Lmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbMzcuODE4Mjc4MDcwMTMyMDMsIC0xMjIuMzQyMDYzNjE2MTkyMjldLCAiem9vbSI6IDEzfQ=='}

# Quickly get unique stops Statewide

In [39]:
from calitp_data_analysis import gcs_geopandas
gcsgp = gcs_geopandas.GCSGeoPandas()

In [40]:
stops = gcsgp.read_parquet('gs://calitp-analytics-data/data-analyses/traffic_ops/export/ca_transit_stops_2025-07-16.parquet')

In [41]:
# shared_data = catalog_utils.get_catalog(catalog_name='shared_data_catalog')

In [42]:
# stops = shared_data.ca_transit_stops.read()

In [43]:
shs = stops.query('meters_to_ca_state_highway <= 50')
# shs = stops #  get all not just shs

In [44]:
shs.head(3)

,org_id,agency,stop_id,stop_name,n_routes,route_ids_served,routetypes,n_arrivals,n_hours_in_service,meters_to_ca_state_highway,base64_url,district_name,geometry
36,rec9sI67HygZotDqo,City of Lodi,114,Kettleman & Central (EB),1,2,3,1,1,7.9,aHR0cHM6Ly93d3cubG9kaS5nb3YvRG9jdW1lbnRDZW50ZX...,10 - Stockton,POINT (-121.26434 38.11602)
58,rec9sI67HygZotDqo,City of Lodi,141,Kettleman & Stockton (WB),1,4,3,0,0,8.6,aHR0cHM6Ly93d3cubG9kaS5nb3YvRG9jdW1lbnRDZW50ZX...,10 - Stockton,POINT (-121.26902 38.11601)
93,rec9sI67HygZotDqo,City of Lodi,18,Beckman & Buena Vista,1,2,3,0,0,31.2,aHR0cHM6Ly93d3cubG9kaS5nb3YvRG9jdW1lbnRDZW50ZX...,10 - Stockton,POINT (-121.25896 38.14119)


Where count is greater than 1, agencies share a GTFS feed. We count each stop once per feed to prevent double-counting

In [45]:
agency_groups = shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').agency.unique()
agency_groups = pd.DataFrame(agency_groups).reset_index().rename(columns={'agency':'agency_group'})

In [46]:
df = shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').count()

In [47]:
df = df.reset_index().rename(columns={'agency':'count'})

In [48]:
shs[['base64_url', 'agency']].drop_duplicates().merge(df, on='base64_url').sort_values('count', ascending=False)

,base64_url,agency,count
50,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Ojai,7
45,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,Ventura County Transportation Commission,7
51,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Simi Valley,7
49,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Moorpark,7
48,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,Gold Coast Transit District,7
...,...,...,...
70,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Mountain View Transportation Management Associ...,1
71,aHR0cHM6Ly9zbG8uY29ubmV4aW9uei5uZXQvcnR0L3B1Ym...,San Luis Obispo Regional Transit Authority,1
72,aHR0cHM6Ly9hcHBzLnNhbnRhY2xhcml0YXRyYW5zaXQuY2...,City of Santa Clarita,1
73,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,Livermore-Amador Valley Transit Authority,1


In [49]:
gdf = shs.drop_duplicates(subset=['base64_url', 'stop_id']).merge(agency_groups, on='base64_url')

In [50]:
gdf.agency_group = gdf.agency_group.astype(str)

In [51]:
gdf

,org_id,agency,stop_id,stop_name,n_routes,route_ids_served,routetypes,n_arrivals,n_hours_in_service,meters_to_ca_state_highway,base64_url,district_name,geometry,agency_group
0,rec9sI67HygZotDqo,City of Lodi,114,Kettleman & Central (EB),1,2,3,1,1,7.9,aHR0cHM6Ly93d3cubG9kaS5nb3YvRG9jdW1lbnRDZW50ZX...,10 - Stockton,POINT (-121.26434 38.11602),['City of Lodi']
1,rec9sI67HygZotDqo,City of Lodi,141,Kettleman & Stockton (WB),1,4,3,0,0,8.6,aHR0cHM6Ly93d3cubG9kaS5nb3YvRG9jdW1lbnRDZW50ZX...,10 - Stockton,POINT (-121.26902 38.11601),['City of Lodi']
2,rec9sI67HygZotDqo,City of Lodi,18,Beckman & Buena Vista,1,2,3,0,0,31.2,aHR0cHM6Ly93d3cubG9kaS5nb3YvRG9jdW1lbnRDZW50ZX...,10 - Stockton,POINT (-121.25896 38.14119),['City of Lodi']
3,rec9sI67HygZotDqo,City of Lodi,208,Kettleman & Central (WB),1,4,3,0,0,4.4,aHR0cHM6Ly93d3cubG9kaS5nb3YvRG9jdW1lbnRDZW50ZX...,10 - Stockton,POINT (-121.26528 38.11621),['City of Lodi']
4,rec9sI67HygZotDqo,City of Lodi,41,Kettleman & Crescent (EB),1,4,3,1,1,5.8,aHR0cHM6Ly93d3cubG9kaS5nb3YvRG9jdW1lbnRDZW50ZX...,10 - Stockton,POINT (-121.28271 38.11532),['City of Lodi']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6788,reccEj7tecw0n60FO,City of Ripon,2511791,Sisk (Applebees) Target Shopping Ctr,1,12600,3,0,0,46.3,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,10 - Stockton,POINT (-121.06266 37.69166),['City of Ripon']
6789,reccEj7tecw0n60FO,City of Ripon,2511792,Sisk (Best Buy) Target Shopping Ctr,1,12600,3,4,4,46.1,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,10 - Stockton,POINT (-121.06579 37.69429),['City of Ripon']
6790,recXIO3srSe6F77pC,Mission Bay Transportation Management Agency,2335513,Berry Street and King Street,1,6283,3,26,12,12.8,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,04 - Oakland,POINT (-122.39839 37.77176),['Mission Bay Transportation Management Agency']
6791,recXIO3srSe6F77pC,Mission Bay Transportation Management Agency,2335510,Berry Street and King Street,1,6280,3,31,10,11.7,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,04 - Oakland,POINT (-122.39842 37.77190),['Mission Bay Transportation Management Agency']
